# Imports

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
import json
import csv
import pandas as pd
import numpy as np 
import os
from scipy.stats import zscore          

import collections
from sklearn import preprocessing

import shutil
import os


from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import confusion_matrix, classification_report

%matplotlib inline
import matplotlib.pyplot as plt

from matplotlib.pyplot import figure, show
from sklearn.model_selection import train_test_split
from sklearn import metrics
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


## **Converting json to csv file**

In [0]:
outfile = open("./business_json.tsv", 'w')
sfile = csv.writer(outfile, delimiter ="\t", quoting=csv.QUOTE_MINIMAL)
sfile.writerow(['business_id', 'stars', 'review_count'])
with open('./drive/My Drive/215/Project1/yelp_dataset/business.json', encoding="utf-8") as f:
   for line in f:
      row = json.loads(line)
# some special char must be encoded in 'utf-8'
      sfile.writerow([row['business_id'], row['stars'], (row['review_count'])])
outfile.close()


In [0]:
df_business = pd.read_csv('business_json.tsv', delimiter ="\t", encoding="utf-8")

In [0]:
df_business.head()

,business_id,stars,review_count
0,1SWheh84yJXfytovILXOAQ,3.0,5
1,QXAEGFB4oINsVuTFxEYKFQ,2.5,128
2,gnKjwL_1w79qoiV3IC_xQQ,4.0,170
3,xvX2CttrVhyG2z1dFg_0xw,5.0,3
4,HhyxOkGAM07SRYtlQ4wMFQ,4.0,4


In [0]:
outfile = open("./review_json.tsv", 'w')
sfile = csv.writer(outfile, delimiter ="\t", quoting=csv.QUOTE_MINIMAL)
sfile.writerow(['business_id','stars', 'text'])
with open('./drive/My Drive/215/Project1//review.json', encoding="utf-8") as f:
  for line in f:
    row = json.loads(line)
# some special char must be encoded in 'utf-8'
    sfile.writerow([row['business_id'], row['stars'], (row['text']).encode('utf-8')])
outfile.close()


In [0]:
df_review= pd.read_csv('review_json.tsv', delimiter ="\t", encoding="utf-8")

In [0]:
df_review.head()

,business_id,stars,text
0,ujmEBvifdJM6h6RLv4wQIg,1.0,b'Total bill for this horrible service? Over $...
1,NZnhc2sEQy3RmzKTZnqtwQ,5.0,"b""I *adore* Travis at the Hard Rock's new Kell..."
2,WTqjgwHlXbSFevF32_DJVw,5.0,"b""I have to say that this office really has it..."
3,ikCg8xy5JIg_NGPx-MSIDA,5.0,"b""Went in for a lunch. Steak sandwich was deli..."
4,b1b1eb3uo-w561D0ZfCEiQ,1.0,b'Today was my second out of three sessions I ...


In [0]:
outfile = open("./tip_json.tsv", 'w')
sfile = csv.writer(outfile, delimiter ="\t", quoting=csv.QUOTE_MINIMAL)
sfile.writerow(['business_id', 'compliment_count', 'text'])
with open('./drive/My Drive/215/Project1/yelp_dataset/tip.json', encoding="utf-8") as f:
   for line in f:
      row = json.loads(line)
# some special char must be encoded in 'utf-8'
      sfile.writerow([row['business_id'], row['compliment_count'], (row['text']).encode('utf-8')])
outfile.close()

In [0]:
df_tip=pd.read_csv('tip_json.tsv', delimiter ="\t", encoding="utf-8")

In [15]:
df_tip.head()

,business_id,compliment_count,text
0,VaKXUpmWTTWDKbpJ3aQdMw,0,"b'Great for watching games, ufc, and whatever ..."
1,OPiPeoJiv92rENwbq76orA,0,b'Happy Hour 2-4 daily with 1/2 price drinks a...
2,5KheTjYPu1HcQzQFtm4_vw,0,b'Good chips and salsa. Loud at times. Good se...
3,TkoyGi8J7YFjA6SbaRzrxg,0,b'The setting and decoration here is amazing. ...
4,AkL6Ous6A1atZejfZXn1Bg,0,b'Molly is definately taking a picture with Sa...


# Data Preprocessing

In [0]:
df_review_agg = df_review.groupby('business_id')['text'].sum() # Grouping reviews by business_id

In [0]:
df_ready_for_sklearn = pd.DataFrame({'business_id': df_review_agg.index, 'all_reviews': df_review_agg.values})

In [7]:
df_ready_for_sklearn.head()

,all_reviews,business_id
0,b'If you are looking for authentic Mexican str...,--1UhMGODdWsrMastO9DZw
1,"b""They have the best Chinese BBQ Pork (Char Si...",--6MefnULPED_I942VcFNA
2,"b""I'm a tad reluctant to write a review as I r...",--7zmmkVg-IMGaXbuVd0SQ
3,b'Dr. LaCognata is great. He is a great liste...,--8LPVSo5i0Oo61X01sV9A
4,b'Always great customer service. Affordable ha...,--9QQLMTbFzLJ_oT-ON3Xw


In [0]:
df_tip_agg = df_tip.groupby('business_id')['text'].sum()

In [0]:
df_ready_for_sklearn_tips = pd.DataFrame({'business_id': df_tip_agg.index, 'all_tips': df_tip_agg.values }) 

In [21]:
df_ready_for_sklearn_tips.head()

,all_tips,business_id
0,"b""Delicious! One of the best burritos, salsa a...",--1UhMGODdWsrMastO9DZw
1,b'BBQ pork is sold out early on Saturday'b'$5 ...,--6MefnULPED_I942VcFNA
2,"b""Good place to stop when traffic don't go.""b'...",--7zmmkVg-IMGaXbuVd0SQ
3,"b""Tip? Don't get your hair cut here. If they'r...",--9QQLMTbFzLJ_oT-ON3Xw
4,b'Wine pairings for my prix fixe meal. Mmmm'b...,--9e1ONYQuAa-CB_Rrw7Tw


In [0]:
df_br = pd.merge(df_business, df_ready_for_sklearn, on= 'business_id', how= 'inner')

In [23]:
df_br.head()

,business_id,stars,review_count,all_reviews
0,1SWheh84yJXfytovILXOAQ,3.0,5,"b""Don't go here if you expect consistent or re..."
1,QXAEGFB4oINsVuTFxEYKFQ,2.5,128,b'My girlfriend and I went for dinner at Emera...
2,gnKjwL_1w79qoiV3IC_xQQ,4.0,170,b'Husband was craving Chicken Teriyaki & gyoza...
3,xvX2CttrVhyG2z1dFg_0xw,5.0,3,b'The associates at this agency are absolutely...
4,HhyxOkGAM07SRYtlQ4wMFQ,4.0,4,"b""Great customer service and great job every t..."


In [0]:
df_merge= pd.merge(df_br, df_ready_for_sklearn_tips, on='business_id', how= 'inner')

In [0]:
#df_tip_final= pd.merge(df_ready_for_sklearn,df_tip_final1, on='business_id')

In [25]:
df_merge.head()

,business_id,stars,review_count,all_reviews,all_tips
0,1SWheh84yJXfytovILXOAQ,3.0,5,"b""Don't go here if you expect consistent or re...",b'Played for the first there this past weekend...
1,QXAEGFB4oINsVuTFxEYKFQ,2.5,128,b'My girlfriend and I went for dinner at Emera...,"b'Spring & egg rolls are great, get them with ..."
2,gnKjwL_1w79qoiV3IC_xQQ,4.0,170,b'Husband was craving Chicken Teriyaki & gyoza...,"b'Tonkatsu Ramen!!!'b'Ramen, Gyoza, miso soup ..."
3,68dUKd8_8liJ7in4aWOSEA,2.5,3,b'horrible. do not go. staff is dishonest and...,b'Good service'
4,5JucpCfHZltJh5r1JabjDg,3.5,7,b'OK place- I\'ve gone for a couple of years a...,b'George does the best gel nails!!!'


In [13]:
df_merge.shape

(123996, 5)

In [12]:
df_final = df_merge[df_merge.review_count >=50]  
df_final.head()

,business_id,stars,review_count,all_reviews,all_tips
1,QXAEGFB4oINsVuTFxEYKFQ,2.5,128,b'My girlfriend and I went for dinner at Emera...,"b'Spring & egg rolls are great, get them with ..."
2,gnKjwL_1w79qoiV3IC_xQQ,4.0,170,b'Husband was craving Chicken Teriyaki & gyoza...,"b'Tonkatsu Ramen!!!'b'Ramen, Gyoza, miso soup ..."
17,KWywu2tTEPWmR9JnBc0WyQ,4.0,107,"b""It's awesome here!!The guys are soooo hot an...","b""Omg I loved it. First time this past weekend..."
19,tstimHoMcYbkSC4eBA1wEg,4.5,184,"b""We found out about this gem from The Man's c...",b'Best most authentic Mexican food in Las Vega...
22,iojTeSaoPuxm4WeCzDUA6w,4.5,52,"b""I got into a parking lot accident with my 20...",b'Really bad service didnt fix my car on time ...


In [16]:
df_final.shape

(26870, 5)

# Normalization & Vectorization

In [0]:
df_mix = df_final.copy()
df_multinb = df_final.copy()

In [33]:
df_mix.head()

,business_id,stars,review_count,all_reviews,all_tips
1,QXAEGFB4oINsVuTFxEYKFQ,2.5,128,b'My girlfriend and I went for dinner at Emera...,"b'Spring & egg rolls are great, get them with ..."
2,gnKjwL_1w79qoiV3IC_xQQ,4.0,170,b'Husband was craving Chicken Teriyaki & gyoza...,"b'Tonkatsu Ramen!!!'b'Ramen, Gyoza, miso soup ..."
17,KWywu2tTEPWmR9JnBc0WyQ,4.0,107,"b""It's awesome here!!The guys are soooo hot an...","b""Omg I loved it. First time this past weekend..."
19,tstimHoMcYbkSC4eBA1wEg,4.5,184,"b""We found out about this gem from The Man's c...",b'Best most authentic Mexican food in Las Vega...
22,iojTeSaoPuxm4WeCzDUA6w,4.5,52,"b""I got into a parking lot accident with my 20...",b'Really bad service didnt fix my car on time ...


In [34]:
df_multinb.head()

,business_id,stars,review_count,all_reviews,all_tips
1,QXAEGFB4oINsVuTFxEYKFQ,2.5,128,b'My girlfriend and I went for dinner at Emera...,"b'Spring & egg rolls are great, get them with ..."
2,gnKjwL_1w79qoiV3IC_xQQ,4.0,170,b'Husband was craving Chicken Teriyaki & gyoza...,"b'Tonkatsu Ramen!!!'b'Ramen, Gyoza, miso soup ..."
17,KWywu2tTEPWmR9JnBc0WyQ,4.0,107,"b""It's awesome here!!The guys are soooo hot an...","b""Omg I loved it. First time this past weekend..."
19,tstimHoMcYbkSC4eBA1wEg,4.5,184,"b""We found out about this gem from The Man's c...",b'Best most authentic Mexican food in Las Vega...
22,iojTeSaoPuxm4WeCzDUA6w,4.5,52,"b""I got into a parking lot accident with my 20...",b'Really bad service didnt fix my car on time ...


In [0]:
df_multinb = df_multinb.drop(['review_count'], axis=1)

In [0]:
#df_multinb.head()

In [0]:
#normalize review_count in range 0-1

df_norm_minmax=(df_final['review_count']-df_final['review_count'].min())/(df_final['review_count'].max()-df_final['review_count'].min())

In [37]:
df_norm_minmax.shape

(26870,)

In [0]:
df_multinb = df_multinb.join(df_norm_minmax)

In [39]:
df_multinb.head()

,business_id,stars,all_reviews,all_tips,review_count
1,QXAEGFB4oINsVuTFxEYKFQ,2.5,b'My girlfriend and I went for dinner at Emera...,"b'Spring & egg rolls are great, get them with ...",0.009400
2,gnKjwL_1w79qoiV3IC_xQQ,4.0,b'Husband was craving Chicken Teriyaki & gyoza...,"b'Tonkatsu Ramen!!!'b'Ramen, Gyoza, miso soup ...",0.014461
17,KWywu2tTEPWmR9JnBc0WyQ,4.0,"b""It's awesome here!!The guys are soooo hot an...","b""Omg I loved it. First time this past weekend...",0.006869
19,tstimHoMcYbkSC4eBA1wEg,4.5,"b""We found out about this gem from The Man's c...",b'Best most authentic Mexican food in Las Vega...,0.016148
22,iojTeSaoPuxm4WeCzDUA6w,4.5,"b""I got into a parking lot accident with my 20...",b'Really bad service didnt fix my car on time ...,0.000241


In [0]:
# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd

In [18]:
encode_numeric_zscore(df_mix,'review_count')
df_mix.head()

,business_id,stars,review_count,all_reviews,all_tips
1,QXAEGFB4oINsVuTFxEYKFQ,2.5,-0.159115,b'My girlfriend and I went for dinner at Emera...,"b'Spring & egg rolls are great, get them with ..."
2,gnKjwL_1w79qoiV3IC_xQQ,4.0,0.005593,b'Husband was craving Chicken Teriyaki & gyoza...,"b'Tonkatsu Ramen!!!'b'Ramen, Gyoza, miso soup ..."
17,KWywu2tTEPWmR9JnBc0WyQ,4.0,-0.241469,"b""It's awesome here!!The guys are soooo hot an...","b""Omg I loved it. First time this past weekend..."
19,tstimHoMcYbkSC4eBA1wEg,4.5,0.060495,"b""We found out about this gem from The Man's c...",b'Best most authentic Mexican food in Las Vega...
22,iojTeSaoPuxm4WeCzDUA6w,4.5,-0.457157,"b""I got into a parking lot accident with my 20...",b'Really bad service didnt fix my car on time ...


In [0]:
# TfIdfVectorizer: transforms text into a "sparse matrix" where rows are text and columns are words, and values are the tf-dif values.
#TF-IDF Vectorization 'all_reviews' text
import sklearn.feature_extraction.text as sk_text

vectorizer_rev = sk_text.TfidfVectorizer(stop_words='english', max_features = 500, min_df=1)

#max_features:  build a vocabulary that only consider the top max_features features ordered by term frequency across the corpus.

rev_matrix = vectorizer_rev.fit_transform(df_mix['all_reviews'])

print(type(rev_matrix))          # Compressed Sparse Row matrix
print(rev_matrix.toarray())        #  convert it to numpy array

In [0]:
#TF-IDF Vectorization 'all_tips' text

vectorizer_tip = sk_text.TfidfVectorizer(stop_words='english', max_features = 500, min_df=1)

#max_features:  build a vocabulary that only consider the top max_features features ordered by term frequency across the corpus.

tip_matrix = vectorizer_tip.fit_transform(df_mix['all_tips'])

print(type(tip_matrix))          # Compressed Sparse Row matrix
print(tip_matrix.toarray())        #  convert it to numpy array

In [0]:
# Converting resultant outputs of normalization and vectorization into numpy arrays
rev_numpyArr = rev_matrix.toarray()
tip_numpyArr = tip_matrix.toarray()
reviewCountArr = df_mix.as_matrix(columns=['review_count']) #z score

In [22]:
rev_numpyArr.shape

(26870, 500)

In [23]:
tip_numpyArr.shape

(26870, 500)

In [48]:
reviewCountArr.shape

(26870, 1)

In [0]:
reviewCountArr_mnb = df_multinb.as_matrix(columns=['review_count']) #min - max.shape
reviewCountArr_mnb.shape

In [0]:
X = np.concatenate((rev_numpyArr, tip_numpyArr, reviewCountArr), axis =1)       # features used for prediction
y = np.array(df_mix.stars)                                   # target

In [0]:
mnb_X = np.concatenate((rev_numpyArr, tip_numpyArr, reviewCountArr_mnb), axis =1)

In [0]:
x_withouttips= np.concatenate((rev_numpyArr, reviewCountArr_mnb), axis =1)

In [0]:
from sklearn import preprocessing                             # label encoding y to use in classification models

le = preprocessing.LabelEncoder()
le_y=le.fit_transform(y)
le.classes_

# Train Test Splits

In [0]:
from sklearn.model_selection import train_test_split #classification sykit

## For Classification Models (Except Multinomial Naive Bayes)

In [0]:
X_train_sy, X_test_sy, Y_train_sy, Y_test_sy = train_test_split(X, le_y, test_size=0.2, random_state=1 )

## For Multinomial Naive Bayes Model

In [0]:
X_train_sym, X_test_sym, Y_train_sym, Y_test_sym = train_test_split(mnb_X, le_y, test_size=0.2, random_state=1 )

for classification  model without tips

In [0]:
X_train_wt, X_test_wt, Y_train_wt, Y_test_wt = train_test_split(x_withouttips, le_y, test_size=0.2, random_state=1 )

## For Linear Regression Model

In [0]:
X_train_syreg, X_test_syreg, y_train_syreg, y_test_syreg = train_test_split(X, y, test_size=0.2, random_state=1)  #Regression sykit

linear regression  model without tips

In [0]:
X_train_regtw, X_test_regwt, y_train_regwt, y_test_regwt = train_test_split(x_withouttips, y, test_size=0.2, random_state=1) 

# Regression Model from 1st Project

## Linear regression Scikit 

In [63]:
#Linear Regression
from sklearn import  linear_model
preprocess = False
li = linear_model.LinearRegression()
li.fit(X_train_syreg, y_train_syreg)
li_prediction = li.predict(X_test_syreg)
print('\n')
print ('Linear Regression')
print('\n')
print('Actual values:')
print('\n')
print(y_test_syreg[:15])
print('\n')
print('Predicted values:')
print('\n')
print(li_prediction[:15])
    



Linear Regression


Actual values:


[3.  3.  4.  4.  3.  3.  2.  4.  4.  1.5 3.  4.  3.5 4.5 4. ]


Predicted values:


[3.31878733 2.73642956 4.10404199 4.15132647 3.30057546 2.71920539
 2.48502125 4.32037824 3.9487685  2.25583581 3.38094065 3.81370488
 3.50917125 4.32361708 3.75810964]


In [64]:
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(y_test_syreg, li_prediction))
print("Final score (RMSE): {}".format(score))

Final score (RMSE): 0.2633016708390385


In [67]:
#Measure R2 Score
score = metrics.r2_score(y_test_syreg, li_prediction)
print("Final score (R2): {}".format(score))

Final score (R2): 0.8553701422760607


# Classification Models From 1st Project

## K Nearest Neighbors

In [68]:
#Nearest neighbor                                                         3/15 matches

neigh = KNeighborsClassifier()
neigh.fit(X_train_sy, Y_train_sy)
neigh_prediction = neigh.predict(X_test_sy)
print ('Nearest Neighbor')
print('\n')
print('confusion matrix:')
print(confusion_matrix(Y_test_sy, neigh_prediction))
print('\n')
print('classification report:')
print(classification_report(Y_test_sy, neigh_prediction))
print('Actual values:')
print(Y_test_sy[:15])
print('Predicted values:')
print(neigh_prediction[:15])

Nearest Neighbor


confusion matrix:
[[  0   3   2   0   0   0   0   0   1]
 [  4  10  14   7   1   2   1   5   1]
 [  0  11  24  27  14  20   4  10   1]
 [  0   6  20  71  82  81  21  18   0]
 [  0   2  17  73 196 292 137  39   1]
 [  1   0   4  26 153 591 500  81   4]
 [  0   0   4  11  72 449 826 220   9]
 [  0   0   1   2  20 140 413 380  35]
 [  0   0   1   0   3  10  29 130  41]]


classification report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.31      0.22      0.26        45
           2       0.28      0.22      0.24       111
           3       0.33      0.24      0.28       299
           4       0.36      0.26      0.30       757
           5       0.37      0.43      0.40      1360
           6       0.43      0.52      0.47      1591
           7       0.43      0.38      0.41       991
           8       0.44      0.19      0.27       214

   micro avg       0.40      0.40      0.40 

## Logistic Regression

In [76]:
#Logistic Regression                                        # 8/15 matches

clf_lr = LogisticRegression()                                 # clf short for classifier
clf_lr.fit(X_train_sy, Y_train_sy)
prediction_lr = clf_lr.predict(X_test_sy)
print('\n')
print ('Logistic Regression')
print('\n')
print('confusion matrix:')
print(confusion_matrix(Y_test_sy, prediction_lr))
print('\n')
print('classification report:')
print(classification_report(Y_test_sy, prediction_lr))
print('Actual values:')
print(Y_test_sy[:15])
print('Predicted values:')
print(prediction_lr[:15])
preprocess = False

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)




Logistic Regression


confusion matrix:
[[   0    5    1    0    0    0    0    0    0]
 [   0   10   14   15    5    0    0    1    0]
 [   0    3   10   54   37    7    0    0    0]
 [   0    3    3   57  164   68    2    2    0]
 [   0    2    2   33  248  404   68    0    0]
 [   0    2    1    6   83  756  494   17    1]
 [   0    0    0    1   10  240 1175  159    6]
 [   0    0    0    0    0   13  392  549   37]
 [   0    0    0    0    0    0    2  153   59]]


classification report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.40      0.22      0.29        45
           2       0.32      0.09      0.14       111
           3       0.34      0.19      0.25       299
           4       0.45      0.33      0.38       757
           5       0.51      0.56      0.53      1360
           6       0.55      0.74      0.63      1591
           7       0.62      0.55      0.59       991
           8 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Multinomial Naive Bayes

In [78]:
# Multinomial Naive Bayes model                                # 2/15 matches

clf = MultinomialNB()
clf.fit(X_train_sym, Y_train_sym)
prediction = clf.predict(X_test_sym)
print('\n')
print ('MultinomialNB')
print('\n')
print('confusion matrix:')
print(confusion_matrix(Y_test_sym, prediction))
print('\n')
print('classification report:')
print(classification_report(Y_test_sym, prediction))
print('Actual values:')
print(Y_test_sym[:15])
print('Predicted values:')
print(prediction[:15])



MultinomialNB


confusion matrix:
[[  0   6   0   0   0   0   0   0   0]
 [  0  19  12   8   2   2   0   1   1]
 [  0  10  33  23  23  19   1   0   2]
 [  0   7  43  30  56 117  13   7  26]
 [  0   4  50  48  50 424  99  25  57]
 [  0   1  17  36  53 546 536  65 106]
 [  0   2   4  14  24 209 978 179 181]
 [  0   0   3   1   0  13 486 215 273]
 [  0   0   0   0   0   0  19  34 161]]


classification report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.39      0.42      0.40        45
           2       0.20      0.30      0.24       111
           3       0.19      0.10      0.13       299
           4       0.24      0.07      0.10       757
           5       0.41      0.40      0.41      1360
           6       0.46      0.61      0.53      1591
           7       0.41      0.22      0.28       991
           8       0.20      0.75      0.32       214

   micro avg       0.38      0.38      0.38  

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Support Vector Machine

In [79]:
#SVM                                                                     10/15 matches
from sklearn.svm import SVC

svm_clf = SVC(kernel='linear')
svm_clf.fit(X_train_sy, Y_train_sy)
svm_prediction = svm_clf.predict(X_test_sy)
print('\n')
print ('SVM')
print('\n')
print('confusion matrix:')
print(confusion_matrix(Y_test_sy, svm_prediction))
print('\n')
print('classification report:')
print(classification_report(Y_test_sy, svm_prediction))
print('Actual values:')
print(Y_test_sy[:15])
print('Predicted values:')
print(svm_prediction[:15])



SVM


confusion matrix:
[[   0    5    1    0    0    0    0    0    0]
 [   1   22   18    4    0    0    0    0    0]
 [   0   11   40   57    3    0    0    0    0]
 [   0    1   19  156  118    5    0    0    0]
 [   0    0    5   60  455  230    7    0    0]
 [   0    0    1    8  108  961  279    3    0]
 [   0    0    0    0    3  216 1247  124    1]
 [   0    0    0    0    0    4  272  682   33]
 [   0    0    0    0    0    0    1  115   98]]


classification report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.56      0.49      0.52        45
           2       0.48      0.36      0.41       111
           3       0.55      0.52      0.53       299
           4       0.66      0.60      0.63       757
           5       0.68      0.71      0.69      1360
           6       0.69      0.78      0.73      1591
           7       0.74      0.69      0.71       991
           8       0.74      

# Data Preprocessing for Tensorflow

In [0]:
df_tf = pd.DataFrame(X) # for classification
df_tf_wt = pd.DataFrame(x_withouttips) # for classification without tips

In [0]:
df_tf["stars"]= np.array(df_final.stars) #add stars col to df 
df_tf_wt["stars"]= np.array(df_final.stars) #add stars col to df_tf_wt

In [0]:
df_tf_reg = df_tf.copy() # for regression
df_tf_wt_reg = df_tf_wt.copy() # for regression  without tips

In [0]:
df_tf['stars']= df_tf['stars'].astype(np.str) #obj type compatibility by conv to string
df_tf.head()

In [0]:
df_tf_wt['stars']= df_tf_wt['stars'].astype(np.str) #obj type compatibility by conv to string
df_tf_wt.head()

## **to_xy function**

In [0]:
# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column. 
    target_type = df[target].dtypes
    target_type = target_type[0] if isinstance(target_type, collections.Sequence) else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32, np.object):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    else:
        # Regression
        return df[result].values.astype(np.float32), df[target].values.astype(np.float32)

In [0]:
#input for Tensorflow Classification Model
x,y = to_xy(df_tf,"stars") #classification  x y
x.shape

In [0]:
#input for Tensorflow Regression Model
x_reg,y_reg= to_xy(df_tf_reg,"stars") # reg x y
x_reg.shape

In [41]:
#input for Tensorflow Classification Model without tips
x_wt,y_wt = to_xy(df_tf_wt,"stars") #classification  x y
x_wt.shape

(26870, 501)

In [42]:
#input for Tensorflow Regression Model without tips
x_reg_wt,y_reg_wt= to_xy(df_tf_wt_reg,"stars") # reg x y
x_reg_wt.shape

(26870, 501)

## Tensorflow Regression Model without tips

In [0]:
x_trainreg, x_testreg, y_trainreg, y_testreg = train_test_split(x_reg, y_reg, test_size=0.20, random_state=50)

In [0]:
#regression model activation = relu and optimizer = adam
from keras import optimizers
save_path = './gdrive/My Drive/215/Project1/'
model = Sequential()
for i in range (5):
  checkpointer = ModelCheckpoint(filepath="./gdrive/My Drive/215/Project1/best_weightsreg.hdf5", verbose=0, save_best_only=True)
  model.add(Dense(50, input_dim=x.shape[1], activation='relu')) # Hidden 1      
  model.add(Dense(25, activation='relu')) # Hidden 2
  model.add(Dense(1)) # Output
  adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

  model.compile(loss='mean_squared_error', optimizer='adam')
  monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
  
  model.fit(x_trainreg,y_trainreg, validation_data=(x_testreg,y_testreg),callbacks=[monitor],verbose=2,epochs=1000)
  model.save(os.path.join(save_path, 'best_weightsreg.hdf5'))
 
  pred_reg = model.predict(x_testreg)
# Measure MSE error.  
  score_reg = metrics.mean_squared_error(pred_reg,y_testreg)
  print("Final score (MSE): {}".format(score_reg))
# Measure RMSE error.  RMSE is common for regression.
  score_reg = np.sqrt(metrics.mean_squared_error(pred_reg,y_testreg))
  print("Final score (RMSE): {}".format(score_reg))

Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 13s - loss: 0.6456 - val_loss: 0.1043
Epoch 2/1000
 - 4s - loss: 0.0836 - val_loss: 0.0761
Epoch 3/1000
 - 5s - loss: 0.0649 - val_loss: 0.0612
Epoch 4/1000
 - 4s - loss: 0.0571 - val_loss: 0.0601
Epoch 5/1000
 - 4s - loss: 0.0540 - val_loss: 0.0553
Epoch 6/1000
 - 4s - loss: 0.0527 - val_loss: 0.0575
Epoch 7/1000
 - 4s - loss: 0.0514 - val_loss: 0.0553
Epoch 8/1000
 - 4s - loss: 0.0501 - val_loss: 0.0536
Epoch 9/1000
 - 4s - loss: 0.0493 - val_loss: 0.0572
Epoch 10/1000
 - 4s - loss: 0.0487 - val_loss: 0.0626
Epoch 11/1000
 - 4s - loss: 0.0471 - val_loss: 0.0530
Epoch 12/1000
 - 4s - loss: 0.0465 - val_loss: 0.0528
Epoch 13/1000
 - 4s - loss: 0.0455 - val_loss: 0.0520
Epoch 14/1000
 - 4s - loss: 0.0446 - val_loss: 0.0526
Epoch 15/1000
 - 4s - loss: 0.0437 - val_loss: 0.0549
Epoch 16/1000
 - 4s - loss: 0.0424 - val_loss: 0.0522
Epoch 17/1000
 - 4s - loss: 0.0424 - val_loss: 0.0602
Epoch 18/1000
 - 4s - loss: 0.0419 - val_

In [0]:
from keras.models import load_model
model = load_model(os.path.join(save_path, 'best_weightsreg.hdf5'))
pred_reg = model.predict(x_testreg)
  # Measure MSE error.  
score_reg = metrics.mean_squared_error(pred_reg,y_testreg)
print("Final score (MSE): {}".format(score_reg))
  # Measure RMSE error.  RMSE is common for regression.
score_reg = np.sqrt(metrics.mean_squared_error(pred_reg,y_testreg))
print("Final score (RMSE): {}".format(score_reg))

Final score (MSE): 0.07432857155799866
Final score (RMSE): 0.2726326584815979


In [0]:
from keras import optimizers
save_path = './gdrive/My Drive/215/Project1/'
model = Sequential()
for i in range (5):
  checkpointer = ModelCheckpoint(filepath="./gdrive/My Drive/215/Project1/best_weightsreg.hdf5", verbose=0, save_best_only=True)
  model.add(Dense(50, input_dim=x.shape[1], activation='sigmoid')) # Hidden 1      
  model.add(Dense(25, activation='sigmoid')) # Hidden 2
  model.add(Dense(1)) # Output
  adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

  model.compile(loss='mean_squared_error', optimizer='adam')
  monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
  
  model.fit(x_trainreg,y_trainreg, validation_data=(x_testreg,y_testreg),callbacks=[monitor],verbose=2,epochs=1000)
  
  pred_reg = model.predict(x_testreg)
# Measure MSE error.  
  score_reg = metrics.mean_squared_error(pred_reg,y_testreg)
  print("Final score (MSE): {}".format(score_reg))
# Measure RMSE error.  RMSE is common for regression.
  score_reg = np.sqrt(metrics.mean_squared_error(pred_reg,y_testreg))
  print("Final score (RMSE): {}".format(score_reg))

Instructions for updating:
Use tf.cast instead.
Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 5s - loss: 1.2029 - val_loss: 0.4491
Epoch 2/1000
 - 3s - loss: 0.3363 - val_loss: 0.1788
Epoch 3/1000
 - 3s - loss: 0.1190 - val_loss: 0.0871
Epoch 4/1000
 - 3s - loss: 0.0786 - val_loss: 0.0719
Epoch 5/1000
 - 3s - loss: 0.0690 - val_loss: 0.0665
Epoch 6/1000
 - 3s - loss: 0.0642 - val_loss: 0.0627
Epoch 7/1000
 - 3s - loss: 0.0606 - val_loss: 0.0601
Epoch 8/1000
 - 3s - loss: 0.0572 - val_loss: 0.0593
Epoch 9/1000
 - 3s - loss: 0.0547 - val_loss: 0.0549
Epoch 10/1000
 - 3s - loss: 0.0528 - val_loss: 0.0543
Epoch 11/1000
 - 3s - loss: 0.0518 - val_loss: 0.0529
Epoch 12/1000
 - 3s - loss: 0.0507 - val_loss: 0.0522
Epoch 13/1000
 - 3s - loss: 0.0498 - val_loss: 0.0524
Epoch 14/1000
 - 3s - loss: 0.0493 - val_loss: 0.0513
Epoch 15/1000
 - 3s - loss: 0.0488 - val_loss: 0.0519
Epoch 16/1000
 - 3s - loss: 0.0486 - val_loss: 0.0508
Epoch 17/1000
 - 3s - loss: 0.0480 - val_loss: 0

In [0]:
from keras.models import load_model
model = load_model(os.path.join(save_path, 'best_weightsreg.hdf5'))
pred_reg = model.predict(x_testreg)
  # Measure MSE error.  
score_reg = metrics.mean_squared_error(pred_reg,y_testreg)
print("Final score (MSE): {}".format(score_reg))
  # Measure RMSE error.  RMSE is common for regression.
score_reg = np.sqrt(metrics.mean_squared_error(pred_reg,y_testreg))
print("Final score (RMSE): {}".format(score_reg))

Final score (MSE): 0.055805038660764694
Final score (RMSE): 0.23623089492321014


In [0]:
save_path = './gdrive/My Drive/215/Project1/'
model = Sequential()
for i in range (5):
  checkpointer = ModelCheckpoint(filepath="./gdrive/My Drive/215/Project1/best_weightsreg.hdf5", verbose=0, save_best_only=True)
  model.add(Dense(100, input_dim=x.shape[1], activation='tanh')) # Hidden 1      
  model.add(Dense(60, activation='tanh')) # Hidden 2
  model.add(Dense(1)) # Output
  adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

  model.compile(loss='mean_squared_error', optimizer='adam')
  monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
  
  model.fit(x_trainreg,y_trainreg, validation_data=(x_testreg,y_testreg),callbacks=[monitor],verbose=2,epochs=1000)
  
  pred_reg = model.predict(x_testreg)
# Measure MSE error.  
  score_reg = metrics.mean_squared_error(pred_reg,y_testreg)
  print("Final score (MSE): {}".format(score_reg))
# Measure RMSE error.  RMSE is common for regression.
  score_reg = np.sqrt(metrics.mean_squared_error(pred_reg,y_testreg))
  print("Final score (RMSE): {}".format(score_reg))

Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 3s - loss: 0.3427 - val_loss: 0.0768
Epoch 2/1000
 - 3s - loss: 0.0649 - val_loss: 0.0587
Epoch 3/1000
 - 3s - loss: 0.0580 - val_loss: 0.0587
Epoch 4/1000
 - 3s - loss: 0.0546 - val_loss: 0.0535
Epoch 5/1000
 - 3s - loss: 0.0532 - val_loss: 0.0633
Epoch 6/1000
 - 3s - loss: 0.0523 - val_loss: 0.0530
Epoch 7/1000
 - 3s - loss: 0.0513 - val_loss: 0.0505
Epoch 8/1000
 - 3s - loss: 0.0494 - val_loss: 0.0522
Epoch 9/1000
 - 3s - loss: 0.0495 - val_loss: 0.0529
Epoch 10/1000
 - 3s - loss: 0.0484 - val_loss: 0.0502
Epoch 11/1000
 - 3s - loss: 0.0474 - val_loss: 0.0530
Epoch 12/1000
 - 3s - loss: 0.0475 - val_loss: 0.0497
Epoch 00012: early stopping
Final score (MSE): 0.04965855926275253
Final score (RMSE): 0.2228420078754425
Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 4s - loss: 0.0738 - val_loss: 0.0518
Epoch 2/1000
 - 4s - loss: 0.0477 - val_loss: 0.0550
Epoch 3/1000
 - 4s - loss: 0.0471 - val_loss: 0.052

In [0]:
from keras.models import load_model
model = load_model(os.path.join(save_path, 'best_weightsreg.hdf5'))
pred_reg = model.predict(x_testreg)
  # Measure MSE error.  
score_reg = metrics.mean_squared_error(pred_reg,y_testreg)
print("Final score (MSE): {}".format(score_reg))
  # Measure RMSE error.  RMSE is common for regression.
score_reg = np.sqrt(metrics.mean_squared_error(pred_reg,y_testreg))
print("Final score (RMSE): {}".format(score_reg))

Final score (MSE): 0.055805038660764694
Final score (RMSE): 0.23623089492321014


In [0]:
#regression model 
from keras import optimizers
save_path = './gdrive/My Drive/215/Project1/'
model = Sequential()
for i in range (5):
  checkpointer = ModelCheckpoint(filepath="./gdrive/My Drive/215/Project1/best_weightsreg.hdf5", verbose=0, save_best_only=True)
  model.add(Dense(100, input_dim=x.shape[1], activation='sigmoid')) # Hidden 1      
  model.add(Dense(50, activation='sigmoid')) # Hidden 2
  model.add(Dense(1)) # Output
  sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
  model.compile(loss='mean_squared_error', optimizer='sgd')
  monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
  
  model.fit(x_trainreg,y_trainreg, validation_data=(x_testreg,y_testreg),callbacks=[monitor],verbose=2,epochs=1000)

  pred_reg = model.predict(x_testreg)
# Measure MSE error.  
  score_reg = metrics.mean_squared_error(pred_reg,y_testreg)
  print("Final score (MSE): {}".format(score_reg))
# Measure RMSE error.  RMSE is common for regression.
  score_reg = np.sqrt(metrics.mean_squared_error(pred_reg,y_testreg))
  print("Final score (RMSE): {}".format(score_reg))

Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 4s - loss: 0.5111 - val_loss: 0.4792
Epoch 2/1000
 - 2s - loss: 0.4902 - val_loss: 0.4897
Epoch 3/1000
 - 2s - loss: 0.4880 - val_loss: 0.4744
Epoch 4/1000
 - 2s - loss: 0.4856 - val_loss: 0.4752
Epoch 5/1000
 - 2s - loss: 0.4825 - val_loss: 0.4694
Epoch 6/1000
 - 2s - loss: 0.4802 - val_loss: 0.4666
Epoch 7/1000
 - 2s - loss: 0.4749 - val_loss: 0.4656
Epoch 8/1000
 - 2s - loss: 0.4698 - val_loss: 0.4552
Epoch 9/1000
 - 2s - loss: 0.4637 - val_loss: 0.4543
Epoch 10/1000
 - 2s - loss: 0.4561 - val_loss: 0.4442
Epoch 11/1000
 - 2s - loss: 0.4451 - val_loss: 0.4312
Epoch 12/1000
 - 2s - loss: 0.4302 - val_loss: 0.4155
Epoch 13/1000
 - 2s - loss: 0.4112 - val_loss: 0.3886
Epoch 14/1000
 - 2s - loss: 0.3852 - val_loss: 0.3616
Epoch 15/1000
 - 2s - loss: 0.3513 - val_loss: 0.3218
Epoch 16/1000
 - 2s - loss: 0.3087 - val_loss: 0.2777
Epoch 17/1000
 - 2s - loss: 0.2609 - val_loss: 0.2396
Epoch 18/1000
 - 2s - loss: 0.2151 - val_l

In [0]:
from keras.models import load_model
model = load_model(os.path.join(save_path, 'best_weightsreg.hdf5'))
pred_reg = model.predict(x_testreg)
  # Measure MSE error.  
score_reg = metrics.mean_squared_error(pred_reg,y_testreg)
print("Final score (MSE): {}".format(score_reg))
  # Measure RMSE error.  RMSE is common for regression.
score_reg = np.sqrt(metrics.mean_squared_error(pred_reg,y_testreg))
print("Final score (RMSE): {}".format(score_reg))

Final score (MSE): 0.055805038660764694
Final score (RMSE): 0.23623089492321014


In [0]:
from keras import optimizers
save_path = './gdrive/My Drive/215/Project1/'
model = Sequential()
for i in range (5):
  checkpointer = ModelCheckpoint(filepath="./gdrive/My Drive/215/Project1/best_weightsreg.hdf5", verbose=0, save_best_only=True)
  model.add(Dense(100, input_dim=x.shape[1], activation='tanh')) # Hidden 1      
  model.add(Dense(55, activation='tanh')) # Hidden 2
  model.add(Dense(1)) # Output
  sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
  model.compile(loss='mean_squared_error', optimizer='sgd')
  monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
  
  model.fit(x_trainreg,y_trainreg, validation_data=(x_testreg,y_testreg),callbacks=[monitor],verbose=2,epochs=1000)
  model.save(os.path.join(save_path, 'best_weightsreg.hdf5'))
  
  pred_reg = model.predict(x_testreg)
# Measure MSE error.  
  score_reg = metrics.mean_squared_error(pred_reg,y_testreg)
  print("Final score (MSE): {}".format(score_reg))
# Measure RMSE error.  RMSE is common for regression.
  score_reg = np.sqrt(metrics.mean_squared_error(pred_reg,y_testreg))
  print("Final score (RMSE): {}".format(score_reg))

Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 5s - loss: 0.3635 - val_loss: 0.1443
Epoch 2/1000
 - 3s - loss: 0.1189 - val_loss: 0.1051
Epoch 3/1000
 - 3s - loss: 0.1000 - val_loss: 0.0939
Epoch 4/1000
 - 3s - loss: 0.0928 - val_loss: 0.0877
Epoch 5/1000
 - 3s - loss: 0.0883 - val_loss: 0.0868
Epoch 6/1000
 - 3s - loss: 0.0853 - val_loss: 0.0858
Epoch 7/1000
 - 3s - loss: 0.0828 - val_loss: 0.0828
Epoch 8/1000
 - 3s - loss: 0.0806 - val_loss: 0.0826
Epoch 9/1000
 - 3s - loss: 0.0790 - val_loss: 0.0767
Epoch 10/1000
 - 3s - loss: 0.0776 - val_loss: 0.0765
Epoch 11/1000
 - 3s - loss: 0.0766 - val_loss: 0.0775
Epoch 12/1000
 - 3s - loss: 0.0755 - val_loss: 0.0802
Epoch 13/1000
 - 3s - loss: 0.0747 - val_loss: 0.0801
Epoch 14/1000
 - 3s - loss: 0.0739 - val_loss: 0.0725
Epoch 15/1000
 - 3s - loss: 0.0733 - val_loss: 0.0742
Epoch 16/1000
 - 3s - loss: 0.0726 - val_loss: 0.0761
Epoch 17/1000
 - 3s - loss: 0.0720 - val_loss: 0.0725
Epoch 18/1000
 - 3s - loss: 0.0715 - val_l

In [0]:
  from keras.models import load_model
  model = load_model(os.path.join(save_path, 'best_weightsreg.hdf5'))

  pred_reg = model.predict(x_testreg)
  # Measure MSE error.  
  score_reg = metrics.mean_squared_error(pred_reg,y_testreg)
  print("Final score (MSE): {}".format(score_reg))
  # Measure RMSE error.  RMSE is common for regression.
  score_reg = np.sqrt(metrics.mean_squared_error(pred_reg,y_testreg))
  print("Final score (RMSE): {}".format(score_reg))

Final score (MSE): 0.07432857155799866
Final score (RMSE): 0.2726326584815979


In [0]:
save_path = './gdrive/My Drive/215/Project1/'
model = Sequential()
checkpointer = ModelCheckpoint(filepath="./gdrive/My Drive/215/Project1/best_weightsreg.hdf5", verbose=0, save_best_only=True)
for i in range (5):
  
  model.add(Dense(100, input_dim=x.shape[1], activation='relu')) # Hidden 1      
  model.add(Dense(60, activation='relu')) # Hidden 2
  model.add(Dense(30, activation='relu')) #Hidden 3
  model.add(Dense(1)) # Output
  sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
  model.compile(loss='mean_squared_error', optimizer='sgd')
  monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
  
  model.fit(x_trainreg,y_trainreg, validation_data=(x_testreg,y_testreg),callbacks=[monitor],verbose=2,epochs=1000)
  
  

  pred_reg = model.predict(x_testreg)
  # Measure MSE error.  
  score_reg = metrics.mean_squared_error(pred_reg,y_testreg)
  print("Final score (MSE): {}".format(score_reg))
  # Measure RMSE error.  RMSE is common for regression.
  score_reg = np.sqrt(metrics.mean_squared_error(pred_reg,y_testreg))
  print("Final score (RMSE): {}".format(score_reg))


Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 6s - loss: 0.3985 - val_loss: 0.1278
Epoch 2/1000
 - 3s - loss: 0.1175 - val_loss: 0.1045
Epoch 3/1000
 - 3s - loss: 0.1011 - val_loss: 0.0992
Epoch 4/1000
 - 3s - loss: 0.0925 - val_loss: 0.1043
Epoch 5/1000
 - 3s - loss: 0.0865 - val_loss: 0.0827
Epoch 6/1000
 - 3s - loss: 0.0811 - val_loss: 0.0766
Epoch 7/1000
 - 3s - loss: 0.0781 - val_loss: 0.1009
Epoch 8/1000
 - 3s - loss: 0.0749 - val_loss: 0.0744
Epoch 9/1000
 - 3s - loss: 0.0706 - val_loss: 0.0722
Epoch 10/1000
 - 3s - loss: 0.0688 - val_loss: 0.0671
Epoch 11/1000
 - 3s - loss: 0.0666 - val_loss: 0.0661
Epoch 12/1000
 - 3s - loss: 0.0647 - val_loss: 0.0629
Epoch 13/1000
 - 3s - loss: 0.0633 - val_loss: 0.0655
Epoch 14/1000
 - 3s - loss: 0.0614 - val_loss: 0.0631
Epoch 15/1000
 - 3s - loss: 0.0608 - val_loss: 0.0635
Epoch 16/1000
 - 3s - loss: 0.0595 - val_loss: 0.0590
Epoch 17/1000
 - 3s - loss: 0.0591 - val_loss: 0.0585
Epoch 18/1000
 - 3s - loss: 0.0575 - val_l

In [0]:
from keras.models import load_model
model = load_model(os.path.join(save_path, 'best_weightsreg.hdf5'))
pred_reg = model.predict(x_testreg)
  # Measure MSE error.  
score_reg = metrics.mean_squared_error(pred_reg,y_testreg)
print("Final score (MSE): {}".format(score_reg))
  # Measure RMSE error.  RMSE is common for regression.
score_reg = np.sqrt(metrics.mean_squared_error(pred_reg,y_testreg))
print("Final score (RMSE): {}".format(score_reg))
     

Final score (MSE): 0.07432857155799866
Final score (RMSE): 0.2726326584815979


## **Classification TF Model without tips**

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=50)

In [0]:
from keras import optimizers
#classification model
checkpointer = ModelCheckpoint(filepath="./gdrive/My Drive/215/Project1/best_weights.hdf5", verbose=0, save_best_only=True)
for i in range (5):
  model = Sequential()

  model.add(Dense(100, input_dim=x.shape[1], activation='relu')) # Hidden 1    
  model.add(Dense(80, activation='relu'))# Hidden 2
  model.add(Dense(20, activation='relu'))# Hidden 2
  model.add(Dense(10, activation='relu'))# Hidden 2
  model.add(Dense(y.shape[1], activation='softmax'))
  adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
  model.compile(loss='categorical_crossentropy', optimizer='adam')
  monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=2, mode='auto') 
  model.fit(x_train, y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpointer],verbose=2,epochs=1000)
  
  pred = model.predict(x_test)

  pred = np.argmax(pred,axis=1) # raw probabilities to chosen class (highest probability)


  print("Prediction: {}".format(pred[0:10]))

  y_true= np.argmax(y_test,axis=1) 

  score = metrics.accuracy_score(y_true, pred)

  print("Accuracy score: {}".format(score))

  score = metrics.precision_score(y_true, pred, average= "weighted")
  print("Precision score: {}".format(score))

  score = metrics.recall_score(y_true, pred, average= "weighted")
  print("Recall score: {}".format(score))

  score = metrics.f1_score(y_true, pred, average= "weighted")
  print("F1 score: {}".format(score))


Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 7s - loss: 1.2164 - val_loss: 0.7539
Epoch 2/1000
 - 4s - loss: 0.6610 - val_loss: 0.6410
Epoch 3/1000
 - 4s - loss: 0.6059 - val_loss: 0.6153
Epoch 4/1000
 - 4s - loss: 0.5804 - val_loss: 0.5978
Epoch 5/1000
 - 4s - loss: 0.5656 - val_loss: 0.6697
Epoch 6/1000
 - 4s - loss: 0.5504 - val_loss: 0.6464
Epoch 00006: early stopping
Prediction: [6 4 5 8 6 5 6 4 4 3]
Accuracy score: 0.7161063908385666
Precision score: 0.7195491400481856
Recall score: 0.7161063908385666
F1 score: 0.7134960478040481


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 7s - loss: 1.2550 - val_loss: 0.7187
Epoch 2/1000
 - 4s - loss: 0.6431 - val_loss: 0.6253
Epoch 3/1000
 - 4s - loss: 0.5948 - val_loss: 0.6219
Epoch 4/1000
 - 4s - loss: 0.5787 - val_loss: 0.5987
Epoch 5/1000
 - 5s - loss: 0.5579 - val_loss: 0.6033
Epoch 6/1000
 - 4s - loss: 0.5485 - val_loss: 0.6074
Epoch 00006: early stopping
Prediction: [7 4 5 8 6 5 7 3 4 3]
Accuracy score: 0.7382711488732915
Precision score: 0.7401043332647281
Recall score: 0.7382711488732915
F1 score: 0.7346291630623031
Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 8s - loss: 1.0413 - val_loss: 0.6876
Epoch 2/1000
 - 5s - loss: 0.6421 - val_loss: 0.6706
Epoch 3/1000
 - 5s - loss: 0.6096 - val_loss: 0.6260
Epoch 4/1000
 - 5s - loss: 0.5830 - val_loss: 0.5995
Epoch 5/1000
 - 5s - loss: 0.5688 - val_loss: 0.6362
Epoch 6/1000
 - 5s - loss: 0.5576 - val_loss: 0.6249
Epoch 00006: early stopping
Prediction: [6 4 5 8 6 5 7 3 4 3]
Accuracy 

In [0]:
from keras.models import load_model
model = load_model(os.path.join(save_path, 'best_weights.hdf5'))
pred = model.predict(x_test)

pred = np.argmax(pred,axis=1) # raw probabilities to chosen class (highest probability)

print("Prediction: {}".format(pred[0:10]))

y_true= np.argmax(y_test,axis=1) 

score = metrics.accuracy_score(y_true, pred)

print("Accuracy score: {}".format(score))

score = metrics.precision_score(y_true, pred, average= "weighted")
print("Precision score: {}".format(score))

score = metrics.recall_score(y_true, pred, average= "weighted")
print("Recall score: {}".format(score))

score = metrics.f1_score(y_true, pred, average= "weighted")
print("F1 score: {}".format(score))


Prediction: [6 4 5 8 6 5 7 4 4 3]
Accuracy score: 0.745474695234577
Precision score: 0.7449756124586652
Recall score: 0.745474695234577
F1 score: 0.7436126096157896


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
checkpointer = ModelCheckpoint(filepath="./gdrive/My Drive/215/Project1/best_weights.hdf5", verbose=0, save_best_only=True)
for i in range (5):
  model = Sequential()

  model.add(Dense(100, input_dim=x.shape[1], activation='tanh')) # Hidden 1    
  model.add(Dense(80, activation='tanh'))# Hidden 2
  model.add(Dense(10, activation='tanh'))# Hidden 2
  model.add(Dense(y.shape[1], activation='softmax'))
  adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
  model.compile(loss='categorical_crossentropy', optimizer='adam')

  #model.fit(x,y,verbose=2,epochs=100)    # Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch.
  monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=2, mode='auto') 
  model.fit(x_train, y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpointer],verbose=2,epochs=1000)
  #model.load_weights('./gdrive/My Drive/215/Project1/best_weights.hdf5')
  
  pred = model.predict(x_test)

  pred = np.argmax(pred,axis=1) # raw probabilities to chosen class (highest probability)


  print("Prediction: {}".format(pred[0:10]))

  y_true= np.argmax(y_test,axis=1) 

  score = metrics.accuracy_score(y_true, pred)

  print("Accuracy score: {}".format(score))

  score = metrics.precision_score(y_true, pred, average= "weighted")
  print("Precision score: {}".format(score))

  score = metrics.recall_score(y_true, pred, average= "weighted")
  print("Recall score: {}".format(score))

  score = metrics.f1_score(y_true, pred, average= "weighted")
  print("F1 score: {}".format(score))


Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 8s - loss: 1.0956 - val_loss: 0.7656
Epoch 2/1000
 - 4s - loss: 0.7046 - val_loss: 0.6708
Epoch 3/1000
 - 4s - loss: 0.6432 - val_loss: 0.6616
Epoch 4/1000
 - 4s - loss: 0.6122 - val_loss: 0.6351
Epoch 5/1000
 - 4s - loss: 0.5934 - val_loss: 0.6236
Epoch 6/1000
 - 4s - loss: 0.5876 - val_loss: 0.6242
Epoch 7/1000
 - 4s - loss: 0.5744 - val_loss: 0.6170
Epoch 8/1000
 - 4s - loss: 0.5672 - val_loss: 0.6289
Epoch 9/1000
 - 4s - loss: 0.5632 - val_loss: 0.6115
Epoch 10/1000
 - 4s - loss: 0.5574 - val_loss: 0.6173
Epoch 11/1000
 - 4s - loss: 0.5561 - val_loss: 0.6171
Epoch 00011: early stopping
Prediction: [6 4 5 8 6 5 6 4 4 4]
Accuracy score: 0.7318064277798301
Precision score: 0.7300889270122873
Recall score: 0.7318064277798301
F1 score: 0.7270669770877425


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 7s - loss: 1.0784 - val_loss: 0.7648
Epoch 2/1000
 - 4s - loss: 0.7104 - val_loss: 0.7350
Epoch 3/1000
 - 4s - loss: 0.6493 - val_loss: 0.6852
Epoch 4/1000
 - 4s - loss: 0.6220 - val_loss: 0.6938
Epoch 5/1000
 - 4s - loss: 0.6060 - val_loss: 0.6367
Epoch 6/1000
 - 4s - loss: 0.5959 - val_loss: 0.6353
Epoch 7/1000
 - 4s - loss: 0.5829 - val_loss: 0.6391
Epoch 8/1000
 - 4s - loss: 0.5775 - val_loss: 0.6703
Epoch 00008: early stopping
Prediction: [7 4 5 8 6 5 7 3 4 3]
Accuracy score: 0.7105652013298854
Precision score: 0.7094860743603939
Recall score: 0.7105652013298854
F1 score: 0.7068653988182214
Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 8s - loss: 1.0762 - val_loss: 0.7818
Epoch 2/1000
 - 4s - loss: 0.7091 - val_loss: 0.6972
Epoch 3/1000
 - 4s - loss: 0.6582 - val_loss: 0.6537
Epoch 4/1000
 - 4s - loss: 0.6285 - val_loss: 0.6685
Epoch 5/1000
 - 4s - loss: 0.6090 - val_loss: 0.6411
Epoch 6/1000
 - 4s

In [0]:
from keras.models import load_model
model = load_model(os.path.join(save_path, 'best_weights.hdf5'))
pred = model.predict(x_test)

pred = np.argmax(pred,axis=1) # raw probabilities to chosen class (highest probability)

print("Prediction: {}".format(pred[0:10]))

y_true= np.argmax(y_test,axis=1) 

score = metrics.accuracy_score(y_true, pred)

print("Accuracy score: {}".format(score))

score = metrics.precision_score(y_true, pred, average= "weighted")
print("Precision score: {}".format(score))

score = metrics.recall_score(y_true, pred, average= "weighted")
print("Recall score: {}".format(score))

score = metrics.f1_score(y_true, pred, average= "weighted")
print("F1 score: {}".format(score))


Prediction: [6 4 5 8 6 5 7 3 4 3]
Accuracy score: 0.7423346878463244
Precision score: 0.7363850449556301
Recall score: 0.7423346878463244
F1 score: 0.7386209172394163


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
checkpointer = ModelCheckpoint(filepath="./gdrive/My Drive/215/Project1/best_weights.hdf5", verbose=0, save_best_only=True)
for i in range (5):
  model = Sequential()

  model.add(Dense(100, input_dim=x.shape[1], activation='relu')) # Hidden 1    
  model.add(Dense(80, activation='sigmoid'))# Hidden 2
  model.add(Dense(20, activation='sigmoid'))# Hidden 2
  model.add(Dense(10, activation='sigmoid'))# Hidden 2
  model.add(Dense(y.shape[1], activation='softmax'))
  adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
  model.compile(loss='categorical_crossentropy', optimizer='adam')

  #model.fit(x,y,verbose=2,epochs=100)    # Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch.
  monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=2, mode='auto') 
  model.fit(x_train, y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpointer],verbose=2,epochs=1000)
  #model.load_weights('./gdrive/My Drive/215/Project1/best_weights.hdf5')
  
  pred = model.predict(x_test)

  pred = np.argmax(pred,axis=1) # raw probabilities to chosen class (highest probability)


  print("Prediction: {}".format(pred[0:10]))

  y_true= np.argmax(y_test,axis=1) 

  score = metrics.accuracy_score(y_true, pred)

  print("Accuracy score: {}".format(score))

  score = metrics.precision_score(y_true, pred, average= "weighted")
  print("Precision score: {}".format(score))

  score = metrics.recall_score(y_true, pred, average= "weighted")
  print("Recall score: {}".format(score))

  score = metrics.f1_score(y_true, pred, average= "weighted")
  print("F1 score: {}".format(score))


Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 9s - loss: 1.5448 - val_loss: 1.1503
Epoch 2/1000
 - 5s - loss: 1.0012 - val_loss: 0.9057
Epoch 3/1000
 - 5s - loss: 0.8468 - val_loss: 0.8230
Epoch 4/1000
 - 4s - loss: 0.7715 - val_loss: 0.7565
Epoch 5/1000
 - 4s - loss: 0.7189 - val_loss: 0.7127
Epoch 6/1000
 - 5s - loss: 0.6791 - val_loss: 0.6904
Epoch 7/1000
 - 4s - loss: 0.6417 - val_loss: 0.6615
Epoch 8/1000
 - 5s - loss: 0.6198 - val_loss: 0.6593
Epoch 9/1000
 - 5s - loss: 0.6029 - val_loss: 0.6423
Epoch 10/1000
 - 5s - loss: 0.5908 - val_loss: 0.6325
Epoch 11/1000
 - 5s - loss: 0.5810 - val_loss: 0.6355
Epoch 12/1000
 - 5s - loss: 0.5716 - val_loss: 0.6278
Epoch 13/1000
 - 5s - loss: 0.5619 - val_loss: 0.6157
Epoch 14/1000
 - 5s - loss: 0.5525 - val_loss: 0.6170
Epoch 15/1000
 - 5s - loss: 0.5448 - val_loss: 0.6155
Epoch 00015: early stopping
Prediction: [6 4 5 8 6 5 7 3 4 3]
Accuracy score: 0.7417805688954562
Precision score: 0.7394339918240317
Recall score: 0.7

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 9s - loss: 1.7612 - val_loss: 1.3832
Epoch 2/1000
 - 5s - loss: 1.1199 - val_loss: 0.9790
Epoch 3/1000
 - 5s - loss: 0.8959 - val_loss: 0.8575
Epoch 4/1000
 - 5s - loss: 0.8030 - val_loss: 0.7869
Epoch 5/1000
 - 5s - loss: 0.7399 - val_loss: 0.7337
Epoch 6/1000
 - 5s - loss: 0.6957 - val_loss: 0.7011
Epoch 7/1000
 - 5s - loss: 0.6595 - val_loss: 0.6874
Epoch 8/1000
 - 5s - loss: 0.6320 - val_loss: 0.6610
Epoch 9/1000
 - 5s - loss: 0.6133 - val_loss: 0.6483
Epoch 10/1000
 - 5s - loss: 0.5923 - val_loss: 0.6466
Epoch 11/1000
 - 5s - loss: 0.5812 - val_loss: 0.6263
Epoch 12/1000
 - 5s - loss: 0.5665 - val_loss: 0.6361
Epoch 13/1000
 - 5s - loss: 0.5546 - val_loss: 0.6229
Epoch 14/1000
 - 5s - loss: 0.5447 - val_loss: 0.6189
Epoch 15/1000
 - 5s - loss: 0.5339 - val_loss: 0.6167
Epoch 16/1000
 - 5s - loss: 0.5259 - val_loss: 0.6299
Epoch 17/1000
 - 5s - loss: 0.5183 - val_loss: 0.6171
Epoch 00017: early stopping
Prediction: [6

In [0]:
from keras.models import load_model
model = load_model(os.path.join(save_path, 'best_weights.hdf5'))
pred = model.predict(x_test)

pred = np.argmax(pred,axis=1) # raw probabilities to chosen class (highest probability)

print("Prediction: {}".format(pred[0:10]))

y_true= np.argmax(y_test,axis=1) 

score = metrics.accuracy_score(y_true, pred)

print("Accuracy score: {}".format(score))

score = metrics.precision_score(y_true, pred, average= "weighted")
print("Precision score: {}".format(score))

score = metrics.recall_score(y_true, pred, average= "weighted")
print("Recall score: {}".format(score))

score = metrics.f1_score(y_true, pred, average= "weighted")
print("F1 score: {}".format(score))

Prediction: [6 4 5 8 6 5 7 3 4 3]
Accuracy score: 0.7523088289619505
Precision score: 0.7467630031293142
Recall score: 0.7523088289619505
F1 score: 0.7489068980078943


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
from keras import optimizers
#classification model
checkpointer = ModelCheckpoint(filepath="./gdrive/My Drive/215/Project1/best_weights.hdf5", verbose=0, save_best_only=True)
for i in range (5):
  model = Sequential()

  model.add(Dense(50, input_dim=x.shape[1], activation='relu')) # Hidden 1    
  model.add(Dense(40, activation='relu'))# Hidden 2
  model.add(Dense(30, activation='relu'))# Hidden 3
  model.add(Dense(10, activation='relu'))# Hidden 4
  model.add(Dense(y.shape[1], activation='softmax'))
  sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
  model.compile(loss='mean_squared_error', optimizer='sgd')

  #model.fit(x,y,verbose=2,epochs=100)    # Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch.
  monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=2, mode='auto') 
  model.fit(x_train, y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpointer],verbose=2,epochs=1000)
  #model.load_weights('./gdrive/My Drive/215/Project1/best_weights.hdf5')
  
  pred = model.predict(x_test)

  pred = np.argmax(pred,axis=1) # raw probabilities to chosen class (highest probability)


  print("Prediction: {}".format(pred[0:10]))

  y_true= np.argmax(y_test,axis=1) 

  score = metrics.accuracy_score(y_true, pred)

  print("Accuracy score: {}".format(score))

  score = metrics.precision_score(y_true, pred, average= "weighted")
  print("Precision score: {}".format(score))

  score = metrics.recall_score(y_true, pred, average= "weighted")
  print("Recall score: {}".format(score))

  score = metrics.f1_score(y_true, pred, average= "weighted")
  print("F1 score: {}".format(score))
#model.load_weights('./gdrive/My Drive/215/Project1/best_weightsreg.hdf5')

Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 8s - loss: 0.0985 - val_loss: 0.0982
Epoch 2/1000
 - 4s - loss: 0.0979 - val_loss: 0.0976
Epoch 3/1000
 - 4s - loss: 0.0974 - val_loss: 0.0971
Epoch 4/1000
 - 4s - loss: 0.0969 - val_loss: 0.0966
Epoch 5/1000
 - 4s - loss: 0.0964 - val_loss: 0.0961
Epoch 6/1000
 - 4s - loss: 0.0959 - val_loss: 0.0956
Epoch 7/1000
 - 4s - loss: 0.0954 - val_loss: 0.0952
Epoch 00007: early stopping
Prediction: [6 6 6 6 6 6 6 6 6 6]
Accuracy score: 0.30051717768747693
Precision score: 0.09039947621493206
Recall score: 0.30051717768747693
F1 score: 0.13898919468045806


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 8s - loss: 0.0980 - val_loss: 0.0975
Epoch 2/1000
 - 4s - loss: 0.0970 - val_loss: 0.0965
Epoch 3/1000
 - 4s - loss: 0.0959 - val_loss: 0.0954
Epoch 4/1000
 - 4s - loss: 0.0948 - val_loss: 0.0943
Epoch 5/1000
 - 4s - loss: 0.0938 - val_loss: 0.0933
Epoch 6/1000
 - 4s - loss: 0.0928 - val_loss: 0.0924
Epoch 7/1000
 - 4s - loss: 0.0919 - val_loss: 0.0916
Epoch 8/1000
 - 4s - loss: 0.0912 - val_loss: 0.0910
Epoch 9/1000
 - 4s - loss: 0.0907 - val_loss: 0.0905
Epoch 10/1000
 - 4s - loss: 0.0902 - val_loss: 0.0901
Epoch 11/1000
 - 4s - loss: 0.0898 - val_loss: 0.0898
Epoch 00011: early stopping
Prediction: [6 6 6 6 6 6 6 6 6 6]
Accuracy score: 0.30070188400443293
Precision score: 0.09042162304381544
Recall score: 0.30070188400443293
F1 score: 0.13903512273763613
Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 8s - loss: 0.0985 - val_loss: 0.0981
Epoch 2/1000
 - 4s - loss: 0.0977 - val_loss: 0.0973
Epoch 3/1000

In [0]:
from keras.models import load_model
model = load_model(os.path.join(save_path, 'best_weights.hdf5'))
pred = model.predict(x_test)

pred = np.argmax(pred,axis=1) # raw probabilities to chosen class (highest probability)

print("Prediction: {}".format(pred[0:10]))

y_true= np.argmax(y_test,axis=1) 

score = metrics.accuracy_score(y_true, pred)

print("Accuracy score: {}".format(score))

score = metrics.precision_score(y_true, pred, average= "weighted")
print("Precision score: {}".format(score))

score = metrics.recall_score(y_true, pred, average= "weighted")
print("Recall score: {}".format(score))

score = metrics.f1_score(y_true, pred, average= "weighted")
print("F1 score: {}".format(score))

Prediction: [6 6 6 6 6 6 6 6 6 6]
Accuracy score: 0.30070188400443293
Precision score: 0.21664624385438536
Recall score: 0.30070188400443293
F1 score: 0.13935806763192468


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
from keras import optimizers
#classification model
checkpointer = ModelCheckpoint(filepath="./gdrive/My Drive/215/Project1/best_weights.hdf5", verbose=0, save_best_only=True)
for i in range (5):
  model = Sequential()

  model.add(Dense(50, input_dim=x.shape[1], activation='tanh')) # Hidden 1    
  model.add(Dense(40, activation='tanh'))# Hidden 2
  model.add(Dense(10, activation='tanh'))# Hidden 3
  model.add(Dense(y.shape[1], activation='softmax'))
  sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
  model.compile(loss='mean_squared_error', optimizer='sgd')

  #model.fit(x,y,verbose=2,epochs=100)    # Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch.
  monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=2, mode='auto') 
  model.fit(x_train, y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpointer],verbose=2,epochs=1000)
  #model.load_weights('./gdrive/My Drive/215/Project1/best_weights.hdf5')
  
  pred = model.predict(x_test)

  pred = np.argmax(pred,axis=1) # raw probabilities to chosen class (highest probability)


  print("Prediction: {}".format(pred[0:10]))

  y_true= np.argmax(y_test,axis=1) 

  score = metrics.accuracy_score(y_true, pred)

  print("Accuracy score: {}".format(score))

  score = metrics.precision_score(y_true, pred, average= "weighted")
  print("Precision score: {}".format(score))

  score = metrics.recall_score(y_true, pred, average= "weighted")
  print("Recall score: {}".format(score))

  score = metrics.f1_score(y_true, pred, average= "weighted")
  print("F1 score: {}".format(score))


Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 8s - loss: 0.0976 - val_loss: 0.0967
Epoch 2/1000
 - 3s - loss: 0.0957 - val_loss: 0.0950
Epoch 3/1000
 - 3s - loss: 0.0942 - val_loss: 0.0937
Epoch 4/1000
 - 3s - loss: 0.0930 - val_loss: 0.0926
Epoch 5/1000
 - 3s - loss: 0.0920 - val_loss: 0.0916
Epoch 6/1000
 - 3s - loss: 0.0911 - val_loss: 0.0908
Epoch 7/1000
 - 3s - loss: 0.0904 - val_loss: 0.0902
Epoch 8/1000
 - 3s - loss: 0.0898 - val_loss: 0.0896
Epoch 9/1000
 - 3s - loss: 0.0893 - val_loss: 0.0892
Epoch 10/1000
 - 3s - loss: 0.0889 - val_loss: 0.0888
Epoch 00010: early stopping
Prediction: [5 6 6 6 5 6 6 6 6 6]
Accuracy score: 0.30144070927225713
Precision score: 0.14579625796592063
Recall score: 0.30144070927225713
F1 score: 0.16986861449800425


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 9s - loss: 0.0981 - val_loss: 0.0974
Epoch 2/1000
 - 4s - loss: 0.0966 - val_loss: 0.0960
Epoch 3/1000
 - 4s - loss: 0.0953 - val_loss: 0.0948
Epoch 4/1000
 - 3s - loss: 0.0941 - val_loss: 0.0937
Epoch 5/1000
 - 3s - loss: 0.0931 - val_loss: 0.0928
Epoch 6/1000
 - 3s - loss: 0.0922 - val_loss: 0.0921
Epoch 7/1000
 - 4s - loss: 0.0916 - val_loss: 0.0915
Epoch 8/1000
 - 4s - loss: 0.0911 - val_loss: 0.0910
Epoch 9/1000
 - 4s - loss: 0.0906 - val_loss: 0.0906
Epoch 10/1000
 - 4s - loss: 0.0903 - val_loss: 0.0903
Epoch 00010: early stopping
Prediction: [6 6 6 6 6 6 6 6 6 6]
Accuracy score: 0.30070188400443293
Precision score: 0.09042162304381544
Recall score: 0.30070188400443293
F1 score: 0.13903512273763613
Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 9s - loss: 0.0970 - val_loss: 0.0958
Epoch 2/1000
 - 4s - loss: 0.0949 - val_loss: 0.0941
Epoch 3/1000
 - 4s - loss: 0.0934 - val_loss: 0.0927
Epoch 4/1000


In [0]:
from keras.models import load_model
model = load_model(os.path.join(save_path, 'best_weights.hdf5'))
pred = model.predict(x_test)

pred = np.argmax(pred,axis=1) # raw probabilities to chosen class (highest probability)

print("Prediction: {}".format(pred[0:10]))

y_true= np.argmax(y_test,axis=1) 

score = metrics.accuracy_score(y_true, pred)

print("Accuracy score: {}".format(score))

score = metrics.precision_score(y_true, pred, average= "weighted")
print("Precision score: {}".format(score))

score = metrics.recall_score(y_true, pred, average= "weighted")
print("Recall score: {}".format(score))

score = metrics.f1_score(y_true, pred, average= "weighted")
print("F1 score: {}".format(score))

Prediction: [5 6 6 6 5 6 6 6 6 6]
Accuracy score: 0.30144070927225713
Precision score: 0.14579625796592063
Recall score: 0.30144070927225713
F1 score: 0.16986861449800425


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
from keras import optimizers
#classification model
checkpointer = ModelCheckpoint(filepath="./gdrive/My Drive/215/Project1/best_weights.hdf5", verbose=0, save_best_only=True)
for i in range (5):
  model = Sequential()

  model.add(Dense(50, input_dim=x.shape[1], activation='sigmoid')) # Hidden 1    
  model.add(Dense(30, activation='sigmoid'))# Hidden 3
  model.add(Dense(10, activation='sigmoid'))# Hidden 4
  model.add(Dense(y.shape[1], activation='softmax'))
  sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
  model.compile(loss='mean_squared_error', optimizer='sgd')

  #model.fit(x,y,verbose=2,epochs=100)    # Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch.
  monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=2, mode='auto') 
  model.fit(x_train, y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpointer],verbose=2,epochs=1000)
  #model.load_weights('./gdrive/My Drive/215/Project1/best_weights.hdf5')
  
  pred = model.predict(x_test)

  pred = np.argmax(pred,axis=1) # raw probabilities to chosen class (highest probability)


  print("Prediction: {}".format(pred[0:10]))

  y_true= np.argmax(y_test,axis=1) 

  score = metrics.accuracy_score(y_true, pred)

  print("Accuracy score: {}".format(score))

  score = metrics.precision_score(y_true, pred, average= "weighted")
  print("Precision score: {}".format(score))

  score = metrics.recall_score(y_true, pred, average= "weighted")
  print("Recall score: {}".format(score))

  score = metrics.f1_score(y_true, pred, average= "weighted")
  print("F1 score: {}".format(score))
#model.load_weights('./gdrive/My Drive/215/Project1/best_weightsreg.hdf5')

Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 10s - loss: 0.1060 - val_loss: 0.1045
Epoch 2/1000
 - 4s - loss: 0.1030 - val_loss: 0.1018
Epoch 3/1000
 - 4s - loss: 0.1006 - val_loss: 0.0996
Epoch 4/1000
 - 4s - loss: 0.0985 - val_loss: 0.0977
Epoch 5/1000
 - 4s - loss: 0.0968 - val_loss: 0.0962
Epoch 6/1000
 - 4s - loss: 0.0954 - val_loss: 0.0950
Epoch 7/1000
 - 4s - loss: 0.0943 - val_loss: 0.0940
Epoch 8/1000
 - 4s - loss: 0.0935 - val_loss: 0.0932
Epoch 9/1000
 - 4s - loss: 0.0928 - val_loss: 0.0926
Epoch 10/1000
 - 4s - loss: 0.0922 - val_loss: 0.0921
Epoch 11/1000
 - 4s - loss: 0.0917 - val_loss: 0.0916
Epoch 12/1000
 - 4s - loss: 0.0913 - val_loss: 0.0911
Epoch 00012: early stopping
Prediction: [6 6 6 6 6 6 6 6 6 6]
Accuracy score: 0.30070188400443293
Precision score: 0.09042162304381544
Recall score: 0.30070188400443293
F1 score: 0.13903512273763613


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 10s - loss: 0.0985 - val_loss: 0.0979
Epoch 2/1000
 - 4s - loss: 0.0971 - val_loss: 0.0965
Epoch 3/1000
 - 4s - loss: 0.0958 - val_loss: 0.0953
Epoch 4/1000
 - 4s - loss: 0.0947 - val_loss: 0.0943
Epoch 5/1000
 - 4s - loss: 0.0938 - val_loss: 0.0934
Epoch 6/1000
 - 4s - loss: 0.0930 - val_loss: 0.0927
Epoch 7/1000
 - 4s - loss: 0.0923 - val_loss: 0.0921
Epoch 8/1000
 - 4s - loss: 0.0917 - val_loss: 0.0915
Epoch 9/1000
 - 4s - loss: 0.0912 - val_loss: 0.0911
Epoch 10/1000
 - 4s - loss: 0.0908 - val_loss: 0.0907
Epoch 00010: early stopping
Prediction: [6 6 6 6 6 6 6 6 6 6]
Accuracy score: 0.30070188400443293
Precision score: 0.09042162304381544
Recall score: 0.30070188400443293
F1 score: 0.13903512273763613
Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 10s - loss: 0.1065 - val_loss: 0.1049
Epoch 2/1000
 - 4s - loss: 0.1037 - val_loss: 0.1023
Epoch 3/1000
 - 4s - loss: 0.1012 - val_loss: 0.1000
Epoch 4/100

In [0]:
from keras.models import load_model
model = load_model(os.path.join(save_path, 'best_weights.hdf5'))
pred = model.predict(x_test)

pred = np.argmax(pred,axis=1) # raw probabilities to chosen class (highest probability)

print("Prediction: {}".format(pred[0:10]))

y_true= np.argmax(y_test,axis=1) 

score = metrics.accuracy_score(y_true, pred)

print("Accuracy score: {}".format(score))

score = metrics.precision_score(y_true, pred, average= "weighted")
print("Precision score: {}".format(score))

score = metrics.recall_score(y_true, pred, average= "weighted")
print("Recall score: {}".format(score))

score = metrics.f1_score(y_true, pred, average= "weighted")
print("F1 score: {}".format(score))

Prediction: [6 6 6 6 6 6 6 6 6 6]
Accuracy score: 0.30070188400443293
Precision score: 0.09042162304381544
Recall score: 0.30070188400443293
F1 score: 0.13903512273763613


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
checkpointer = ModelCheckpoint(filepath="./gdrive/My Drive/215/Project1/best_weights.hdf5", verbose=0, save_best_only=True)
for i in range (5):
  model = Sequential()

  model.add(Dense(70, input_dim=x.shape[1], activation='sigmoid')) # Hidden 1    
  model.add(Dense(40, activation='relu'))# Hidden 3
  model.add(Dense(20, activation='tanh'))# Hidden 4
  model.add(Dense(y.shape[1], activation='softmax'))
  sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
  model.compile(loss='mean_squared_error', optimizer='sgd')

  #model.fit(x,y,verbose=2,epochs=100)    # Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch.
  monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=2, mode='auto') 
  model.fit(x_train, y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpointer],verbose=2,epochs=1000)
  #model.load_weights('./gdrive/My Drive/215/Project1/best_weights.hdf5')
  
  pred = model.predict(x_test)

  pred = np.argmax(pred,axis=1) # raw probabilities to chosen class (highest probability)


  print("Prediction: {}".format(pred[0:10]))

  y_true= np.argmax(y_test,axis=1) 

  score = metrics.accuracy_score(y_true, pred)

  print("Accuracy score: {}".format(score))

  score = metrics.precision_score(y_true, pred, average= "weighted")
  print("Precision score: {}".format(score))

  score = metrics.recall_score(y_true, pred, average= "weighted")
  print("Recall score: {}".format(score))

  score = metrics.f1_score(y_true, pred, average= "weighted")
  print("F1 score: {}".format(score))

Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 9s - loss: 0.0926 - val_loss: 0.0911
Epoch 2/1000
 - 4s - loss: 0.0903 - val_loss: 0.0898
Epoch 3/1000
 - 4s - loss: 0.0894 - val_loss: 0.0892
Epoch 4/1000
 - 4s - loss: 0.0889 - val_loss: 0.0888
Epoch 5/1000
 - 4s - loss: 0.0886 - val_loss: 0.0885
Epoch 6/1000
 - 4s - loss: 0.0884 - val_loss: 0.0883
Epoch 00006: early stopping
Prediction: [6 6 6 6 6 6 6 6 6 6]
Accuracy score: 0.30070188400443293
Precision score: 0.09042162304381544
Recall score: 0.30070188400443293
F1 score: 0.13903512273763613


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 9s - loss: 0.0965 - val_loss: 0.0942
Epoch 2/1000
 - 4s - loss: 0.0930 - val_loss: 0.0918
Epoch 3/1000
 - 4s - loss: 0.0914 - val_loss: 0.0906
Epoch 4/1000
 - 4s - loss: 0.0905 - val_loss: 0.0900
Epoch 5/1000
 - 4s - loss: 0.0899 - val_loss: 0.0895
Epoch 6/1000
 - 4s - loss: 0.0895 - val_loss: 0.0891
Epoch 7/1000
 - 4s - loss: 0.0891 - val_loss: 0.0888
Epoch 00007: early stopping
Prediction: [6 6 6 6 6 6 6 6 6 6]
Accuracy score: 0.30070188400443293
Precision score: 0.09042162304381544
Recall score: 0.30070188400443293
F1 score: 0.13903512273763613
Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 9s - loss: 0.0950 - val_loss: 0.0931
Epoch 2/1000
 - 4s - loss: 0.0918 - val_loss: 0.0911
Epoch 3/1000
 - 4s - loss: 0.0903 - val_loss: 0.0899
Epoch 4/1000
 - 4s - loss: 0.0895 - val_loss: 0.0892
Epoch 5/1000
 - 4s - loss: 0.0889 - val_loss: 0.0888
Epoch 6/1000
 - 4s - loss: 0.0886 - val_loss: 0.0885
Epoch 7/1000
 

In [0]:
from keras.models import load_model
model = load_model(os.path.join(save_path, 'best_weights.hdf5'))
pred = model.predict(x_test)

pred = np.argmax(pred,axis=1) # raw probabilities to chosen class (highest probability)

print("Prediction: {}".format(pred[0:10]))

y_true= np.argmax(y_test,axis=1) 

score = metrics.accuracy_score(y_true, pred)

print("Accuracy score: {}".format(score))

score = metrics.precision_score(y_true, pred, average= "weighted")
print("Precision score: {}".format(score))

score = metrics.recall_score(y_true, pred, average= "weighted")
print("Recall score: {}".format(score))

score = metrics.f1_score(y_true, pred, average= "weighted")
print("F1 score: {}".format(score))

Prediction: [6 6 6 6 6 6 6 6 6 6]
Accuracy score: 0.30070188400443293
Precision score: 0.09042162304381544
Recall score: 0.30070188400443293
F1 score: 0.13903512273763613


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
checkpointer = ModelCheckpoint(filepath="./gdrive/My Drive/215/Project1/best_weights.hdf5", verbose=0, save_best_only=True)
for i in range (5):
  model = Sequential()

  model.add(Dense(50, input_dim=x.shape[1], activation='sigmoid')) # Hidden 1    
  model.add(Dense(30, activation='relu'))# Hidden 3
  model.add(Dense(10, activation='tanh'))# Hidden 4
  model.add(Dense(y.shape[1], activation='softmax'))
  adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
  model.compile(loss='categorical_crossentropy', optimizer='adam')

  monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=2, mode='auto') 
  model.fit(x_train, y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpointer],verbose=2,epochs=1000)
  #model.load_weights('./gdrive/My Drive/215/Project1/best_weights.hdf5')
  
  pred = model.predict(x_test)

  pred = np.argmax(pred,axis=1) # raw probabilities to chosen class (highest probability)


  print("Prediction: {}".format(pred[0:10]))

  y_true= np.argmax(y_test,axis=1) 

  score = metrics.accuracy_score(y_true, pred)

  print("Accuracy score: {}".format(score))

  score = metrics.precision_score(y_true, pred, average= "weighted")
  print("Precision score: {}".format(score))

  score = metrics.recall_score(y_true, pred, average= "weighted")
  print("Recall score: {}".format(score))

  score = metrics.f1_score(y_true, pred, average= "weighted")
  print("F1 score: {}".format(score))

Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 11s - loss: 1.5245 - val_loss: 1.0955
Epoch 2/1000
 - 5s - loss: 0.8914 - val_loss: 0.7798
Epoch 3/1000
 - 5s - loss: 0.7237 - val_loss: 0.6963
Epoch 4/1000
 - 5s - loss: 0.6628 - val_loss: 0.6760
Epoch 5/1000
 - 5s - loss: 0.6337 - val_loss: 0.6415
Epoch 6/1000
 - 5s - loss: 0.6133 - val_loss: 0.6409
Epoch 7/1000
 - 5s - loss: 0.6034 - val_loss: 0.6399
Epoch 8/1000
 - 5s - loss: 0.5911 - val_loss: 0.6163
Epoch 9/1000
 - 5s - loss: 0.5856 - val_loss: 0.6192
Epoch 10/1000
 - 5s - loss: 0.5799 - val_loss: 0.6221
Epoch 00010: early stopping
Prediction: [6 4 5 8 6 5 7 3 4 3]
Accuracy score: 0.7330993719985224
Precision score: 0.7282320799970127
Recall score: 0.7330993719985224
F1 score: 0.7291448527296389


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 11s - loss: 1.4986 - val_loss: 1.0658
Epoch 2/1000
 - 5s - loss: 0.8934 - val_loss: 0.7924
Epoch 3/1000
 - 5s - loss: 0.7408 - val_loss: 0.7025
Epoch 4/1000
 - 5s - loss: 0.6736 - val_loss: 0.6802
Epoch 5/1000
 - 5s - loss: 0.6404 - val_loss: 0.6469
Epoch 6/1000
 - 5s - loss: 0.6190 - val_loss: 0.6475
Epoch 7/1000
 - 5s - loss: 0.6051 - val_loss: 0.6683
Epoch 00007: early stopping
Prediction: [6 4 5 8 6 5 6 3 4 3]
Accuracy score: 0.7127816771333579
Precision score: 0.7131964692489622
Recall score: 0.7127816771333579
F1 score: 0.7085492295450052
Train on 21653 samples, validate on 5414 samples
Epoch 1/1000
 - 12s - loss: 1.5585 - val_loss: 1.1826
Epoch 2/1000
 - 5s - loss: 0.9497 - val_loss: 0.8200
Epoch 3/1000
 - 5s - loss: 0.7542 - val_loss: 0.7191
Epoch 4/1000
 - 5s - loss: 0.6797 - val_loss: 0.6667
Epoch 5/1000
 - 5s - loss: 0.6424 - val_loss: 0.6544
Epoch 6/1000
 - 5s - loss: 0.6185 - val_loss: 0.6317
Epoch 7/1000
 - 

In [0]:
from keras.models import load_model
model = load_model(os.path.join(save_path, 'best_weights.hdf5'))
pred = model.predict(x_test)

pred = np.argmax(pred,axis=1) # raw probabilities to chosen class (highest probability)

print("Prediction: {}".format(pred[0:10]))

y_true= np.argmax(y_test,axis=1) 

score = metrics.accuracy_score(y_true, pred)

print("Accuracy score: {}".format(score))

score = metrics.precision_score(y_true, pred, average= "weighted")
print("Precision score: {}".format(score))

score = metrics.recall_score(y_true, pred, average= "weighted")
print("Recall score: {}".format(score))

score = metrics.f1_score(y_true, pred, average= "weighted")
print("F1 score: {}".format(score))

Prediction: [6 4 5 8 6 5 7 3 4 3]
Accuracy score: 0.7537864794975988
Precision score: 0.7536463350648104
Recall score: 0.7537864794975988
F1 score: 0.7531829468053621


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


tips regression# New Section

# Regression and Classification with tip dataset

Regression

In [0]:
x_trainreg, x_testreg, y_trainreg, y_testreg = train_test_split(x_reg, y_reg, test_size=0.20, random_state=50)

In [45]:
from keras import optimizers
save_path = './gdrive/My Drive/215/Project1/'
model = Sequential()
for i in range (5):
  #checkpointer = ModelCheckpoint(filepath="./gdrive/My Drive/215/Project1/best_weights_reg_with_tips.hdf5", verbose=0, save_best_only=True)
  model.add(Dense(50, input_dim=x.shape[1], activation='sigmoid')) # Hidden 1      
  model.add(Dense(25, activation='sigmoid')) # Hidden 2
  model.add(Dense(1)) # Output
  adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

  model.compile(loss='mean_squared_error', optimizer='adam')
  monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
  
  model.fit(x_trainreg,y_trainreg, validation_data=(x_testreg,y_testreg),callbacks=[monitor],verbose=2,epochs=1000)
 # model.load_weights('./gdrive/My Drive/215/Project1/best_weight_sreg_with_tips.hdf5')
  
  pred_reg = model.predict(x_testreg)
# Measure MSE error.  
  score_reg = metrics.mean_squared_error(pred_reg,y_testreg)
  print("Final score (MSE): {}".format(score_reg))
# Measure RMSE error.  RMSE is common for regression.
  score_reg = np.sqrt(metrics.mean_squared_error(pred_reg,y_testreg))
  print("Final score (RMSE): {}".format(score_reg))

Train on 21496 samples, validate on 5374 samples
Epoch 1/1000
 - 4s - loss: 0.8845 - val_loss: 0.2914
Epoch 2/1000
 - 3s - loss: 0.1740 - val_loss: 0.1140
Epoch 3/1000
 - 3s - loss: 0.0974 - val_loss: 0.0844
Epoch 4/1000
 - 3s - loss: 0.0771 - val_loss: 0.0745
Epoch 5/1000
 - 3s - loss: 0.0686 - val_loss: 0.0687
Epoch 6/1000
 - 3s - loss: 0.0630 - val_loss: 0.0627
Epoch 7/1000
 - 3s - loss: 0.0588 - val_loss: 0.0586
Epoch 8/1000
 - 2s - loss: 0.0551 - val_loss: 0.0552
Epoch 9/1000
 - 3s - loss: 0.0527 - val_loss: 0.0545
Epoch 10/1000
 - 3s - loss: 0.0511 - val_loss: 0.0561
Epoch 11/1000
 - 3s - loss: 0.0499 - val_loss: 0.0505
Epoch 12/1000
 - 3s - loss: 0.0489 - val_loss: 0.0513
Epoch 13/1000
 - 3s - loss: 0.0480 - val_loss: 0.0498
Epoch 14/1000
 - 3s - loss: 0.0473 - val_loss: 0.0490
Epoch 15/1000
 - 3s - loss: 0.0466 - val_loss: 0.0489
Epoch 16/1000
 - 3s - loss: 0.0462 - val_loss: 0.0486
Epoch 17/1000
 - 3s - loss: 0.0460 - val_loss: 0.0481
Epoch 18/1000
 - 3s - loss: 0.0455 - val_l

KeyboardInterrupt: ignored

In [0]:
from keras.models import load_model
model = load_model(os.path.join(save_path, 'best_weights_reg_with_tips.hdf5'))
pred_reg = model.predict(x_testreg)
  # Measure MSE error.  
score_reg = metrics.mean_squared_error(pred_reg,y_testreg)
print("Final score (MSE): {}".format(score_reg))
  # Measure RMSE error.  RMSE is common for regression.
score_reg = np.sqrt(metrics.mean_squared_error(pred_reg,y_testreg))
print("Final score (RMSE): {}".format(score_reg))

Classification

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=50)

In [0]:
#checkpointer = ModelCheckpoint(filepath="./gdrive/My Drive/215/Project1/best_weights_cl_with_tips.hdf5", verbose=0, save_best_only=True)
for i in range (5):
  model = Sequential()

  model.add(Dense(50, input_dim=x.shape[1], activation='sigmoid')) # Hidden 1    
  model.add(Dense(30, activation='relu'))# Hidden 3
  model.add(Dense(10, activation='tanh'))# Hidden 4
  model.add(Dense(y.shape[1], activation='softmax'))
  adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
  model.compile(loss='categorical_crossentropy', optimizer='adam')

  monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=2, mode='auto') 
  model.fit(x_train, y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpointer],verbose=2,epochs=1000)
 # model.load_weights('./gdrive/My Drive/215/Project1/best_weights_cl_with_tips.hdf5')
  
  pred = model.predict(x_test)

  pred = np.argmax(pred,axis=1) # raw probabilities to chosen class (highest probability)


  print("Prediction: {}".format(pred[0:10]))

  y_true= np.argmax(y_test,axis=1) 

  score = metrics.accuracy_score(y_true, pred)

  print("Accuracy score: {}".format(score))

  score = metrics.precision_score(y_true, pred, average= "weighted")
  print("Precision score: {}".format(score))

  score = metrics.recall_score(y_true, pred, average= "weighted")
  print("Recall score: {}".format(score))

  score = metrics.f1_score(y_true, pred, average= "weighted")
  print("F1 score: {}".format(score))

In [0]:
from keras.models import load_model
model = load_model(os.path.join(save_path, 'best_weights_cl_with_tips.hdf5'))
pred = model.predict(x_test)

pred = np.argmax(pred,axis=1) # raw probabilities to chosen class (highest probability)

print("Prediction: {}".format(pred[0:10]))

y_true= np.argmax(y_test,axis=1) 

score = metrics.accuracy_score(y_true, pred)

print("Accuracy score: {}".format(score))

score = metrics.precision_score(y_true, pred, average= "weighted")
print("Precision score: {}".format(score))

score = metrics.recall_score(y_true, pred, average= "weighted")
print("Recall score: {}".format(score))

score = metrics.f1_score(y_true, pred, average= "weighted")
print("F1 score: {}".format(score))